#### Define function to return time difference between two datetime variables

In [ ]:
def calculate_time_difference(timestamp1, timestamp2):
    # Convert timestamps to Unix timestamps
    unix_timestamp1 = int(timestamp1.timestamp())
    unix_timestamp2 = int(timestamp2.timestamp())

    # Calculate the time difference in seconds
    time_difference_seconds = abs(unix_timestamp1 - unix_timestamp2)

    # Calculate minutes and seconds
    minutes = (time_difference_seconds // 60) % 60
    seconds = time_difference_seconds % 60

    # Format the result as "MM:SS"
    time_difference_formatted = f"{minutes:02}:{seconds:02}"

    return time_difference_formatted

#### Define function to find highest dated folder from a list of folder names
This block creates a function which takes a folder location as an input and finds the folders in that location with the highest datetime value where the folder name is in the format YYYY-MM-DD HHMMSS.<br>
It returns the datetime in YYYY-MM-DD HH:MM:SS format to pass to a SQL query.

In [ ]:
def get_sql_start_date(file_path):
    # Retrieve a list of folders in a given file_path
    folders = os.listdir(file_path)
    highest_folder = None

    # Initiate loop using the folders list as input
    for folder in folders:
        try:
            # Convert the folder to datetime, with an expected layout. On failure, it will skip this folder
            folder_datetime = datetime.strptime(folder, "%Y-%m-%d %H%M%S")

            # If current folder in the loop is the latest date loaded, it will store it as the highest_folder variable
            if highest_folder is None or folder_datetime > highest_folder:
                highest_folder = folder_datetime

        except:
            continue

    # Convert the highest-dated folder to a format which can be passed to SQL as a string
    sql_start_date = highest_folder.strftime("%Y-%m-%d %H:%M:%S")

    return sql_start_date

#### Define function to send result of Notebook run to Teams webhook

In [ ]:
def report_outcome(notebook_name, error_count, error_message, run_time, webhook_url):
    if error_count == 0:
        print("No errors noted.")
    else:
        # Message for error
        message_text = f"Fabric Notebook executed:  {notebook_name}.<br>Result: Error ❌<br>Error count: {error_count}.<br>Total run time: {run_time}<br>{error_message}"

        # Create the message payload
        message = {"text": message_text}
    
        # Send the message using a POST request
        response = requests.post(webhook_url, data=json.dumps(message), headers={"Content-Type": "application/json"})

        # Check the response status (optional)
        if response.status_code == 200:
            print("Refresh finished & message sent successfully to Teams!")
        else:
            print(f"Failed to send message. Status code: {response.status_code}")